In [53]:
import pandas as pd
import gzip
import nltk
from nltk.collocations import *
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.metrics.spearman import *
import re
import RAKE
from nltk.corpus import stopwords
from collections import Counter, OrderedDict
from tqdm.notebook import tqdm

### Загрузка данных

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

files = getDF('Magazine_Subscriptions_edited.json.gz').dropna(axis=0, subset=['reviewText'])
meta = getDF('meta_Magazine_Subscriptions.json.gz')

### Предлагаемые способы:

1. Обучить реккурентную есть, которая будет классифицировать каждое слово на предмет того, входит ли оно в состав именованой сущности. В качестве данных должен выступать размеченный текст, где будут размечены сами именованные сущности, а также их границы, и в этом заключается основная трудоемкость задачи. Помимо этого, такой подход не позволит разметить вложенные именованные сущности. Однако это data-driven подход и в целом может дать достаточно качественные результаты.

2. Следующий подход основывается на правилах. Он подходит больше для простых именованных сущностей вроде даты, у которой ограниченный список того, как ее можно задать. Для более сложных ИС подход достаточно топорный, можно пропустить много именованных сущностей, а также он предполагает ручной анализ некоторого количества материала, либо наличие метаданных для составления правил. Однако, если материала очень много и весь он единой тематики, подход может подойти.

3. В конце концов, можно выделить ключевые слова и коллокации, затем отфильтровать их вручную и расширить список, подобрав синонимы для дискрипторов.

### Реализация
Большое количество уже выделенных именованных сущностей можно получить из метаданных.

In [3]:
titles = meta['title'].unique().tolist()
titles.remove('<span class="a-size-medium a-color-secondary"')
titles.remove(' Magazines" />')
titles = [re.sub(r' \(\d+?-[a-zA-Z]+?\)', '', t) for t in titles]

Еще более короткие названия можно посмотреть в в брендах/публикаторах: словосочетания N + Magazine содержат название.

In [4]:
brands = meta['brand'].unique().tolist()
brands = [b for b in brands if 'Magazine' in b]
brands_add = [re.sub(' Magazine', '@', b) for b in brands if ' Magazine' in b]
brands_add = [b.split('@')[0] for b in brands]
brands + brands_add

['Reason Magazine',
 'Hearst Magazines',
 "Harper's Magazine",
 'Golf Magazine',
 'Home Business Magazine',
 'Vermont Life Magazine',
 'Natural History Magazine Inc',
 'The Chelsea Magazine Company',
 'F&W (Magazines)',
 'Strand Magazine',
 'Breakthrough Magazine',
 'D Magazine Partners',
 'Dermascope Magazine',
 'New Art Publ/Bomb Magazine',
 'Dirtrag Magazine',
 'Emmanuel Magazine',
 'Hearst Magazines UK',
 'Living Blues Magazine',
 'Man at Arms Magazine',
 'Moorshead Magazines Ltd',
 'Montana Magazine',
 'New Mexico Magazine',
 'Nebraska Life Magazine',
 'St Louis Magazine Llc',
 'Porthole Magazine',
 'Sacramento Magazines Corp',
 'Relix Magazine Inc',
 'Texas Highways Magazine',
 'Signcraft Magazine',
 'South Dakota Magazine',
 'Make-Up Artist Magazine',
 'Universal Magazines Pty Ltd',
 'Hearst Magazines Italia Spa',
 'F&amp;W (Magazines)',
 'Philadelphia Magazine',
 'National Magazine Company Ltd',
 'Reunions/Magazine/Inc',
 'Geographical Magazine Ltd',
 'Bonnier Magazines & Brand

In [5]:
meta_collocations = titles + brands_add

Логично предположить, что в отзывах ключевыми словами будут товары, о которых идет речь. Применим эти алгоритмы, чтобы найти какие-то дескрипторы или более общие упоминания. Алгоритму сложно обработать сразу все тексты, поэтому будем просматривать частями.

In [6]:
tokenizer = RegexpTokenizer(r"[a-zA-Z0-9]+")
lemmatizer = WordNetLemmatizer()
def normalize(text):
    lemmas = []
    for t in tokenizer.tokenize(text):
        lemmas.append(lemmatizer.lemmatize(t).lower())
    return lemmas

In [7]:
stop = stopwords.words('english')
rake = RAKE.Rake(stop)

In [8]:
files.shape

(89656, 12)

In [9]:
tok_texts = files['reviewText'][10000:20000].apply(normalize)
tok_texts = tok_texts.tolist()
tokens = []
for i in range(len(tok_texts)):
    tokens.extend(tok_texts[i])
tokens = ' '.join(tokens)

In [13]:
rake.run(tokens, maxWords=2, minFrequency=10)

KeyboardInterrupt: 

In [10]:
keywords = [
    'magazine',
    'mag',
    'product',
    'magazine subscription',
    'subscription',
    'product',
    'publication',
    'purchase',
    'electronic version',
    'paper version',
    'print edition',
    'kindle edition',
    'reading material'
]

In [11]:
subscription_to = re.findall('subscription to .+?\.', '. '.join(files['reviewText'].tolist()))
len(subscription_to)

1544

Просматривая контексты, удалось увидеть довольно частотное выражение subscription to, после которого, как правило, следует какое-то упоминание журнала. Вытащим эти упоминания.

In [12]:
ners = []
for text in subscription_to:
    text = nltk.word_tokenize(text)
    text = nltk.pos_tag(text)
    flag = False
    ner = []
    for word in text[2:]:
        if not flag:
            ner.append(word[0])
            if word[1] in ['NN', 'NNS', 'NNP', 'PRP']:
                flag=True
        else:
            if word[1] in ['NN', 'NNS', 'NNP', 'PRP']:
                ner.append(word[0])
            else:
                break
    if not flag:
        ner = []
    if ner:
        ners.append(' '.join(ner))

In [13]:
len(ners)

1452

In [14]:
ners = list(set(ners))
len(ners)

791

Отфильтруем по длине.

In [18]:
ners = [ner for ner in ners if len(ner) < 30]

In [20]:
ners

['the paper copy',
 'APR',
 'SI',
 'my sister',
 'Maxim Magazine',
 'the tablet version',
 'this beautiful magazine',
 'cover the issues I',
 'Asimov',
 'Family Tree magazine',
 'my Army son',
 'Gourmet Magazine',
 'arrive because Amazon',
 'deliver to my house',
 'Newsweek',
 'keep me',
 'Food Network',
 'be cancelled about 6 months',
 'this mag FREE',
 'two former New',
 'be the best deal',
 'AG magazine',
 'digital-only , and each time',
 'Philosophy',
 'the USA',
 'my Kindle',
 'Wondertime',
 'NY Review',
 'this wonderful magazine/',
 'Consumers Report',
 'First',
 'a collection agency',
 'start but the first issue',
 'Time Out New York',
 'Quiltmaker',
 'a boring magazine',
 'get a Kindle-copy',
 'Fine Cooking',
 'make up for the lost issues',
 'Backpacker magazine',
 'the website',
 'this incredible mag',
 'OXM',
 'Traditional Home Classics',
 '5 friends',
 'Blade magazine',
 '`` People Magazine',
 'Martha Stewart Living',
 'an great magazine',
 'open the articles',
 'The Christi

In [21]:
all_ners = meta_collocations + keywords + ners
len(all_ners)

987

In [22]:
all_ners = list(set(all_ners))
len(all_ners)

971

### Получение n-грамм

In [23]:
all_ngrams = []
text = ' '.join(files['reviewText'].tolist())
for ner in tqdm(all_ners):
    ner_regex = re.sub(r'\(', r'\\(', ner)
    ner_regex = re.sub(r'\)', r'\\)', ner_regex)
    left = re.findall('([a-zA-Z0-9]+?)\s+?' + ner_regex, text)
    right = re.findall(ner_regex + '\s+?([a-zA-Z0-9]+?)[\s,.!?]', text)
    left = [((i, ner), ner) for i in left]
    right = [((ner, i), ner) for i in right]
    all_ngrams.extend(left)
    all_ngrams.extend(right)

In [33]:
len(all_ngrams)

711002

### Рассчет метрик

Исключим нечастотные коллокации.

In [35]:
sum(ngrams_dict.values())

711002

In [44]:
ngrams_dict = Counter(all_ngrams)
contexts = []
for key, value in ngrams_dict.items():
    if value > 10:
        contexts.extend([list(key[0])] * value)

In [45]:
len(contexts)

562581

In [46]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [47]:
finder = BigramCollocationFinder.from_documents(contexts)

In [48]:
finder.score_ngrams(bigram_measures.likelihood_ratio)[:20]

[(('this', 'mag'), 57874.77684766975),
 (('this', 'magazine'), 36680.84646573786),
 (('the', 'mag'), 26924.285956906922),
 (('it', 'magazine'), 22292.053863407233),
 (('it', 'it'), 20435.852432416163),
 (('magazine', 'it'), 16875.137746479293),
 (('the', 'magazine'), 15305.723245773275),
 (('mag', 'it'), 14401.599794853028),
 (('my', 'subscription'), 13394.138000176503),
 (('love', 'this mag'), 10273.190939286467),
 (('National', 'G'), 9890.977727257688),
 (('This', 'mag'), 9853.915564929604),
 (('magazine', 'me'), 9008.219290612404),
 (('it', 'was'), 8064.994844098647),
 (('it', 'me'), 8000.0905823639605),
 (('magazine', 'this'), 8000.025660753329),
 (('love', 'this magazine'), 7872.88275212774),
 (('in', 'the mail'), 7814.143601390981),
 (('my', 'Kindle'), 7539.185191890404),
 (('it', 'this'), 7492.215796883381)]

In [50]:
finder.score_ngrams(bigram_measures.pmi)[:20]

[(('Biblical', 'Archaeology'), 16.401261585754856),
 (('Ready', 'Mad'), 14.892247938267001),
 (('REAL', 'SI'), 14.547112452218313),
 (('Bloomberg', 'Businessweek'), 14.373780849332748),
 (('Dog', 'Fancy'), 14.346813801732477),
 (('Country', 'Woman'), 14.014238462645613),
 (('been', 'getting the magazine'), 13.779773209008583),
 (('Scientific American', 'Mind'), 13.658757808047223),
 (('this magazine years', 'ago'), 13.65875780804722),
 (('Practical', 'Horse'), 13.625967872929552),
 (('Western', 'Horse'), 13.625967872929552),
 (('Horse', 'Illustrated'), 13.183449637228602),
 (('VERY', 'GO'), 13.08002726103758),
 (('ESPN', 'Insider'), 12.700821867613765),
 (('placing', 'my order'), 12.69231036775825),
 (('placed', 'my order'), 12.692310367758243),
 (('Art', 'Collector'), 12.658757808047223),
 (('Quilting', 'Art'), 12.658757808047222),
 (('Southwest', 'Art'), 12.658757808047216),
 (('Philosophy', 'Now'), 12.570319843379634)]

In [51]:
finder.score_ngrams(bigram_measures.dice)[:20]

[(('Biblical', 'Archaeology'), 1.0),
 (('Real', 'Simple'), 0.8059259259259259),
 (('Bloomberg', 'Businessweek'), 0.691358024691358),
 (('Dog', 'Fancy'), 0.6329113924050633),
 (('Ready', 'Mad'), 0.5769230769230769),
 (('Food', 'Network'), 0.5536480686695279),
 (('Horse', 'Illustrated'), 0.5492957746478874),
 (('this magazine years', 'ago'), 0.5128205128205128),
 (('REAL', 'SI'), 0.4838709677419355),
 (('Family', 'Handyman'), 0.4801670146137787),
 (('Popular', 'Science'), 0.4548736462093863),
 (('Country', 'Woman'), 0.41379310344827586),
 (('placed', 'my order'), 0.411214953271028),
 (('Popular', 'Mechanics'), 0.38178633975481613),
 (('Western', 'Horse'), 0.3669724770642202),
 (('this', 'mag'), 0.34291565073277397),
 (('Practical', 'Horse'), 0.3364485981308411),
 (('ESPN', 'Insider'), 0.33497536945812806),
 (('VERY', 'GO'), 0.3333333333333333),
 (('Fine', 'Woodwork'), 0.31958762886597936)]

В целом вышла какая-то беда. Мне кажется, это может быть связано с тем, что нередко названия журналов сокращаются до слова. Например 'Bloomberg Businessweek' могут назвать просто Businessweek, но сочетание 'Bloomberg Businessweek' будет более частотно, чем прилагательное + 'Bloomberg Businessweek' или прилагательное + Businessweek. Возможно, стоило поработать с шаблонами при выделении контекстов в том числе. Но в целом мне кажется, что pmi и dice справляются лучше, чем likelihood_ratio, потому что вытаскивают более концептуальные коллокации, названия, а не просто общеупотребительные слова и в целом у них выдача довольно похожа.

### Примеры

In [52]:
candidates = ['mag', 'magazine', 'subscription', 'Vogue', 'product']

In [70]:
ordered_ngram_dict = OrderedDict(sorted(ngrams_dict.items(), key=lambda x: x[1], reverse=True))
for candidate in candidates:
    collocs = [' '.join(k[0]) for k in ordered_ngram_dict.keys() if k[1] == candidate][:5]
    print(candidate + '\n' + '\n'.join(collocs))
    print('\n')

mag
this mag
the mag
This mag
a mag
great mag


magazine
this magazine
the magazine
magazine is
magazine for
This magazine


subscription
my subscription
a subscription
the subscription
subscription to
subscription for


Vogue
Teen Vogue
Vogue is
Vogue and
of Vogue
to Vogue


product
the product
product reviews
new product
this product
and product


